# Deep Learning from Scratch
* From Chap04 - 4.2.3 미니매치 학습

### 4.2.3 미니배치 학습

In [1]:
import sys, os
sys.path.append('C:\\Users\\stevelee\\Documents\\30-Days-Challenges\\deep_learning_scratch_master\\deep-learning-from-scratch-master')
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)  # (60000, 784)
print(t_train.shape)  # (60000, 10)  # 정답 위치의 원소만 1이고 나머지는 0인 배열을 얻는다
                                     # 즉 원-핫 인코딩(One-hot-Encoding)

(60000, 784)
(60000, 10)


* 이 훈련 데이터에서 무작위로 10개만 빼내려면 어떻게 해야할까?

In [2]:
x_train.shape

(60000, 784)

In [3]:
# index로 가져오면 될까?
# inedx를 가져오더라도 랜덤하게 가져올 수 있는 방법이 필요할텐데??

Numpy의 np.random.choice() 함수를 쓰면 다음과 같이 간단하게 해결할 수 있다.

In [4]:
import numpy as np

train_size = x_train.shape[0]
batch_size = 10  # batch_size를 10로 설정한 경우
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [5]:
# batch size가 10인 random한 값을 x_train, t_train로부터 추출한다

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

그렇다면 미니배치 같은 배치 데이터를 지원하는 교차 엔트로피 오차는 어떻게 구현할까?<br/>
(배치를 치원하는 Cross Entropy Error(CEE)는 어떻게 구할 수 있을까?)<br/>
앞서 구현한 CEE를 조금 수정하면 된다

미래에서 왔습니다(20.02.05.Wed) <br/>
(배치용) 교차 엔트로피의 코드가 와닿지 않아서 다시 리뷰...! <br/>
cross_entropy_error() 함수를 주의 깊게 다시 살펴보자!

In [6]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)  # arr.size()는 arr에 몇 개의 요소가 들어있는지 알려준다.
                                  # https://www.codeit.kr/questions/6279
    batch_size = y.shape[0]  # 데이터가 1개라면 y.shape[0]의 값은 1이 되겠군!
                             # 데이터의 개수는 y.shape[0]을 통해 알 수 있게 된다
    return -np.sum(t * np.log(y + 1e-7)) / batch_size  # ???
                                                       # batch크기로 나눠 정규화를 하는 것이다!
                                                       # Ureka! 정규화를 하는거였어! 20.02.05.Wed

In [7]:
# 정답 레이블이 원-핫 인코딩이 아니라 '2'나 '7'등 
# 숫자 레이블로 주어졌을 때의 교차 엔트로피 오차는 어떻게 구할 수 있을까? 

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arrange(batch_size), t] + 1e-7)) / batch_size

np.log(y[np.arrange(batch_size), t])에 대해 자세히 알아보자 <br/>
y[np.arrange(batch_size), t]는 각 데이터의 정답 레이블에 해당하는 신경망을 추출한다. <br/>
* Dive Deep
    * 만약 batch_size가 5라면 np.arrange(batch_size)는 [0, 1, 2, 3, 4]인 넘파이 배열을 생성할 것이다. <br/>
    * 이 때 t에는 [2, 7, 0, 9, 4]등과 같은 각각의 데이터에 대한 정답 레이블에 해당하는 신경망의 출력을 추출하게 된다.
    * 따라서 본 예제에서는 
    * y[np.arrange(batch_size), t]에서 [y[0,2], y[1,7], y[2, 0], y[3, 9], y[4, 4]]의 배열이 생성될 것이다.

In [16]:
# y[np.arange(batch_size), t]는 각 데이터의 정답 레이블에 해당하는 신경망의 출력을 추출한다.

### 4.2.5 왜 손실 함수를 설정하는가?

* 정확도는 매개변수의 미소한 변화에는 거의 반응을 보이지 않고, 반응이 있더라도 그 값이 불연속적으로 갑자기 변화한다.

* 정확도를 지표로 사용하지 않는 이유는 '계단 함수'를 활성화 함수로 사용하지 않는 이유와도 들어 맞는다

* 계단 함수는 한순간만 변화를 일으키지만(0일 때) 시그모이드 함수의 미분(접선)은 출력이 연속적으로 변하고 곡선의 기울기도 연속적으로 변한다
* 즉, 시그모이드 함수의 미분은 어느 장소라도 0이 되지 않는다
* 신경망에서 중요한 성질로, 기울기가 0이 되지 않는 덕분에 신경망이 올바르게 학습할 수 있는 것이다

### Numpy 함수정리
* np.random.choice()

In [5]:
# numpy random.choice
# SciPy.org
# https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.choice.html

* numpy.random.choice(a, size=None, replace=True, p=None)
    * Generates a random sample from a given 1-D array